### PDFファイルに名前をつける

In [2]:
from rename_pdf_file.rename_pdf import PdfRename

%load_ext autoreload
%autoreload 2

In [2]:
pdf_rename = PdfRename(file_path="/Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents")
pdf_rename.rename_pdfs()

  0%|          | 0/19 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Renamed and updated metadata: /Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/9.pdf -> /Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/ハウス食品グループ本社株式会社-統合レポート-2024.pdf
  5%|▌         | 1/19 [00:14<04:25, 14.78s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Renamed and updated metadata: /Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/8.pdf -> /Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/サントリーホールディングス-海外データ-2023年.pdf
 11%|█         | 2/19 [00:24<03:22, 11.89s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Renamed and updated metadata: /Users/user/Desktop/GenerativeAI_apps/third_finance_competition/documents/16.pdf -> /Users/user/Deskt

### Llama parser

In [4]:
from pdf_vectorize.create_llama_parse import LlamaParserVectorStore

/Users/user/Desktop/GenerativeAI_apps/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import nest_asyncio
nest_asyncio.apply()

parser_openai = LlamaParserVectorStore(model="OpenAI", save_name="small")
query_engine_openai = parser_openai.create_vector_engine(save=True)

### 提出用

In [6]:
from langchain_core.documents import Document
from langchain.schema.runnable import RunnableLambda
from typing import List
from langchain_openai import ChatOpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

def query_engine_with_docs(query: str) -> List[Document]:
    # LlamaIndexのレスポンスをLangChainのDocument型に変換
    response = query_engine_openai.query(query)

    # レスポンスをDocument型にラップ
    return [Document(
        page_content=response.response,
        metadata={"source": "query_engine"}
    )]

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

retriever_with_docs = RunnableLambda(query_engine_with_docs)

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever_with_docs
)


template = """
You are a financial data analyst at a large investment bank. Your role is to provide ONLY the key information that directly answers the question, without any additional context or analysis.

Instructions:
- Extract ONLY the specific information asked in the question
- If the information cannot be found or is unclear in the given context, respond with "分かりません"
- If you are uncertain about any part of the answer, respond with "分かりません"
- If multiple interpretations are possible, respond with "分かりません"
- For clear answers, provide them in Japanese
- List items with commas (、) as separators
- Do not include any explanations, descriptions, or analysis
- Do not use bullet points or numbering
- Do not add any headers or formatting
- Do not speculate or make assumptions about missing information

Question: {input}
Context: {context}
Provide only the direct answer:"""

prompt = ChatPromptTemplate.from_template(template)


def summarize_to_max_tokens(text: str) -> str:
    import tiktoken
    enc = tiktoken.get_encoding("cl100k_base")
    tokens = enc.encode(text)

    # 既に54トークン以下の場合はそのまま返す
    if len(tokens) <= 54:
        return text

    # 回答が長い場合、カンマ区切りの項目を優先して短縮
    items = text.split("、")
    current_text = ""

    for item in items:
        temp_text = current_text + ("、" if current_text else "") + item
        temp_tokens = enc.encode(temp_text)
        if len(temp_tokens) <= 54:
            current_text = temp_text
        else:
            break

    return current_text

# RAG チェーンの修正
contextual_rag_chain = (
    {"context": compression_retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    | RunnableLambda(summarize_to_max_tokens))


In [ ]:
import pandas as pd

query_path = "/Users/user/Desktop/GenerativeAI_apps/third_finance_competition/query.csv"
qa = pd.read_csv(query_path)

In [ ]:
from tqdm import tqdm
import weave

weave.init(project_name="第3回金融コンペ", description="")

@weave.op()
def process(chain):
    for i, query in tqdm(enumerate(qa["problem"]), total=len(qa["problem"]), position=0):
        response = chain.invoke(query)
        qa.at[i, "answer"] = response

process(contextual_rag_chain)

  0%|          | 0/100 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  1%|          | 1/100 [00:07<12:45,  7.73s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


  2%|▏         | 2/100 [00:12<10:10,  6.22s/it]

...

 98%|█████████▊| 98/100 [12:16<00:12,  6.42s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Retrieval entering 344fcab6-92f0-4a9a-a46d-c5e6a8f56370: TextNode
Retrieving from object TextNode with query パナソニックグループの2023年度の売上高比率が最も高いセグメントは？
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 99%|█████████▉| 99/100 [12:23<00:06,  6.35s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|██████████| 100/100 [12:28<00:00,  7.48s/it]


In [9]:
save_dir = '/Users/user/Desktop/GenerativeAI_apps/third_finance_competition/submit/'
qa[["index", "answer"]].to_csv(save_dir + "predictions.csv", encoding="utf-8", index=False, header=False)